In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import sqlite3

from matplotlib import pyplot as plt

import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Original GOT Experiments

In [ ]:
p_in = 0.9
p_out = 0.1

strategies = [
    {'name': 'GOT', 'color': 'red'},
    {'name': 'L2', 'color': 'blue'},
    {'name': 'random', 'color': 'black'},
    {'name': 'GW', 'color': 'green'},
    {'name': 'fGOT', 'color': 'orange', 'display_name': 'fGOT', 'marker': '+'},
#     {'name': 'QAP', 'color': 'orange', 'display_name': 'QAP', 'marker': '+'},
#     {'name': 'rrmw', 'color': 'purple'},
#     {'name': 'IPFP', 'color': 'orange'},
]

p_values = [0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]

w2_errors = {}
l2_errors = {}
gw_errors = {}
nmi_score = {}
seeds = list(range(0, 25))

connection = sqlite3.connect(f'../results/results_got.db')
cursor = connection.cursor()

for strategy in strategies:
    name = strategy['name']
    w2_errors[name] = []
    l2_errors[name] = []
    gw_errors[name] = []
    nmi_score[name] = []
    for p in p_values:
        cursor.execute(f"SELECT W2_LOSS, L2_LOSS, GW_LOSS, NMI FROM alignment WHERE STRATEGY='{name}' and P_IN='{p_in}' and P_OUT='{p_out}' and p={p};")
        results = cursor.fetchall()
        assert len(results) > 0, f"No results for strategy '{name}'."
        if len(results) < 25:
            print(f"Only {len(results)} successful seeds for strategy '{name}'.")
        w2_errors[name].append(np.mean(results, axis=0)[0])
        l2_errors[name].append(np.mean(results, axis=0)[1])
        gw_errors[name].append(np.mean(results, axis=0)[2])
        nmi_score[name].append(np.mean(results, axis=0)[3])
cursor.close()
connection.close()

max_w2_error = np.max(list(w2_errors.values()))
max_l2_error = np.max(list(l2_errors.values()))
max_gw_error = np.max(list(gw_errors.values()))
# max_nmi_score = np.max(list(nmi_score.values()))

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(11, 2.5), sharey=True)
ax1.set_ylim(0, 1.02)
ax1.set_title('Normalized L2 error')
ax1.set_xlabel('Edge removal probability')
ax2.set_title('Normalized GOT error')
ax2.set_xlabel('Edge removal probability')
ax3.set_title('Normalized GW error')
ax3.set_xlabel('Edge removal probability')
ax4.set_title('NMI')
ax4.set_xlabel('Edge removal probability')
for strategy in strategies:
    name = strategy['name']
    display_name = strategy['display_name'] if 'display_name' in strategy else name
    marker = strategy['marker'] if 'marker' in strategy else '*'
    l2_error = l2_errors[name] / max_l2_error
    ax1.plot(p_values, l2_error, label=display_name, marker=marker, c=strategy['color'], lw=1)
    w2_error = w2_errors[name] / max_w2_error
    ax2.plot(p_values, w2_error, label=display_name, marker=marker, c=strategy['color'], lw=1)
    gw_error = gw_errors[name] / max_gw_error
    ax3.plot(p_values, gw_error, label=display_name, marker=marker, c=strategy['color'], lw=1)
    nmi = nmi_score[name]# / max_nmi_score
    ax4.plot(p_values, nmi, label=display_name, marker=marker, c=strategy['color'], lw=1)
ax1.legend(loc='lower left')

plt.tight_layout()
plt.savefig(f"../plots/got_alignment/pin_{str(p_in).replace('.', '')}_pout_{str(p_out).replace('.', '')}.pdf", bbox_inches='tight')

plt.show()

## fGOT results

In [ ]:
noise = "_noise"
# noise = ""
connection = sqlite3.connect(f'../results/results_fgot.db')
cursor = connection.cursor()
cursor.execute(f"SELECT STRATEGY, FILTER, SEED, P*100, L2_LOSS, APPROX_LOSS, TIME FROM alignment{noise} where STRATEGY != 'random';")
results = cursor.fetchall()
cursor.close()
connection.close()

strategy_name = {
    'GOT': 'GOT',
    'random': 'random',
#     'random-sq': 'random',
    'fGOT-got': 'fGOT $g(L) = L^{\dagger/2}$',
    'fGOT-sq': 'fGOT $g(L) = L^2$',
    'GW': 'GW',
    'L2': 'GOT L2',
    'QAP-sq': '2-opt $g(L) = L^2$',
    'QAP-got': '2-opt $g(L) = L^{\dagger/2}$',
#     'blowup-QAP+-sq': '2-opt $g(L) = L^2$',
    'blowup-QAP-sq': 'FAQ $g(L) = L^2$',
    'blowup-QAP-got': 'FAQ $g(L) = L^{\dagger/2}$',
    'stochastic-fGOT-sq': 'fGOT $g(L) = L^2$',
    'stochastic-fGOT-got': 'fGOT $g(L) = L^{\dagger/2}$',
}

order = {
    'stochastic-fGOT-sq': 0,
    'stochastic-fGOT-got': 1,
    'blowup-QAP-sq': 2,
    'blowup-QAP-got': 3,
    'QAP-sq': 4,
    'QAP-got': 5,
}

data = pd.DataFrame(results, columns=['strategy', 'filter', 'seed', 'p', 'L2 error', 'GOT error', 'time'])
display_names = [strategy_name[f"{name}-{filter}"] for name, filter in data[['strategy', 'filter']].values]
data['name'] = display_names
data['order'] = [order[f"{name}-{filter}"] for name, filter in data[['strategy', 'filter']].values]
data = data.sort_values(by="order")
# data['time'] = np.log(data['time'])
# data = data[data["strategy"].isin(['PstoH', 'P_nv2', 'QAP-sq', 'QAP-got', 'QAP-L', 'random'])]
for name in np.unique(data['name']):
    print(f"{name} : {int(len(data[data['name']==name])/10)} repetitions")

# Plot L2 error
plt.figure(figsize=(9,6))
sns.lineplot(x='p', y='L2 error', hue='name', markers=True, dashes=False, style='name', data=data)
plt.ylabel('L2 distance', fontsize=20)
plt.xlabel('Graph size', fontsize=20)
plt.xticks([10, 20, 40, 60, 80, 100], fontsize=20)
plt.yticks(fontsize=20)
plt.legend(prop={"size":20})
plt.savefig(f'../plots/alignment{noise}_l2_distance.pdf', bbox_inches='tight')
plt.show()

# Plot L2 error
plt.figure(figsize=(9,6))
sns.lineplot(x='p', y='time', hue='name', markers=True, dashes=False, style='name', data=data)
plt.ylabel('Time (s)', fontsize=20) 
plt.xlabel('Graph size', fontsize=20)
plt.xticks([10, 20, 40, 60, 80, 100], fontsize=20)
plt.yticks(fontsize=20)
plt.legend(prop={"size":20})
plt.savefig(f'../plots/alignment{noise}_time.pdf', bbox_inches='tight')
plt.show()

# Plot GOT error
plt.figure(figsize=(9,6))
sns.lineplot(x='p', y='GOT error', hue='name', markers=True, dashes=False, style='name', data=data)
plt.ylabel('GOT distance', fontsize=20)
plt.xlabel('Graph size', fontsize=20)
plt.xticks([10, 20, 40, 60, 80, 100], fontsize=20)
plt.yticks(fontsize=20)
plt.legend(prop={"size":20})
plt.savefig(f'../plots/alignment{noise}_got_distance.pdf', bbox_inches='tight')
plt.show()